In [10]:
%load_ext autoreload
%autoreload 2

from sympy import *
from sympy.tensor import *
import sympy.printing as printing

from variables import *
from structure import *
from functions import *
from latex import *

import utils
from utils import write_obj
from utils import print_all_variables as pa

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# угловые скорости (платформы, вилки, колеса) относительно пола
omega['platform']= lambda i: Derivative(alpha,t)*e['z']
omega['fork']    = lambda i: omega['platform'](i) + Derivative(theta[i],t)*e['z']
omega['wheel']   = lambda i: omega['fork'](i) + Derivative(psi[i],t)*n_wheel(i)

In [49]:
write_obj(omega['platform'], 'angular_velocity_platform', 'угловая скорость платформы')
write_obj(omega['fork'], 'angular_velocity_fork', 'угловая скорость вилки')
write_obj(omega['wheel'], 'angular_velocity_wheel', 'угловая скорость колеса')

In [14]:
# УРАНЕНИЯ ЭЁЛЕРА И ОТСУТСТВИЕ ПРОСКАЛЬЗЫВАНИЯ 
v[fkey(S)] = euler(S, P)
v[fkey(P)] = euler(P, C)
v[fkey(C)] = euler(C, D)
v[fkey(D)] = lambda i: Matrix([0,0,0]) # проскальзывания нет

In [48]:
write_obj(v[fkey(S)], 'point_S_velocity', 'скорость точки S')
write_obj(v[fkey(P)], 'point_P_velocity', 'скорость точки P')
write_obj(v[fkey(C)], 'point_C_velocity', 'скорость точки C')
write_obj(v[fkey(D)], 'point_D_velocity', 'скорость точки D')

In [16]:
#Полученные выражения из связей для nu1 nu2 delta_x delta_y

eq[fkey(delta['x'])] = lambda i: scalar(v[fkey(S)](i), e['x'])
eq[fkey(delta['y'])] = lambda i: scalar(v[fkey(S)](i), e['y'])
eq[fkey(nu[1])]      = lambda i: scalar(v[fkey(S)](i), e['xi'])
eq[fkey(nu[2])]      = lambda i: scalar(v[fkey(S)](i), e['eta'])

In [17]:
# print(eq[fkey(nu[1])](0))
# print(eq[fkey(nu[2])](0))

In [18]:
# Связи через delta_alpha, delta_x, delta_y

eq['f(delta_x,delta_y)'] = lambda i: solve(
                      [Eq(eq[fkey(delta['x'])](i), delta['x']), Eq(eq[fkey(delta['y'])](i), delta['y'])],
                      [Derivative(psi[i],t), Derivative(theta[i],t)],
                      dict=True)[0]; # возвращает словарь с выражениями для diff(psi) и diff(theta)

eq['diff(psi)']   = lambda i: eq['f(delta_x,delta_y)'](i)[Derivative(psi[i],t)].subs(Derivative(alpha,t), delta['alpha'])
eq['diff(theta)'] = lambda i: eq['f(delta_x,delta_y)'](i)[Derivative(theta[i],t)].subs(Derivative(alpha,t), delta['alpha'])

In [47]:
# 6 уравнений связей

# eq['diff(psi)']      x3
# eq['diff(theta)']    x3

write_obj(eq['diff(psi)'], 'diff(psi)', '3 уравнения связей на diff(psi) = ...')
write_obj(eq['diff(theta)'], 'diff(theta)', '3 уравнения связей на diff(theta) = ...')

In [20]:
# Связи через nu_1, nu_2
eq['f(nu1,nu2)'] = lambda i: solve(
                      [Eq(eq[fkey(nu[1])](i), nu[1]), Eq(eq[fkey(nu[2])](i), nu[2])],
                      [Derivative(psi[i],t), Derivative(theta[i],t)],
                      dict=True)[0]; # возвращает словарь с выражениями для diff(psi) и diff(theta)

eq['diff(psi)_nu']   = lambda i: eq['f(nu1,nu2)'](i)[Derivative(psi[i],t)]
eq['diff(theta)_nu'] = lambda i: eq['f(nu1,nu2)'](i)[Derivative(theta[i],t)]

In [46]:
write_obj(eq['diff(psi)_nu'], 'diff(psi)_nu', '3 уравнения связей на diff(psi) = ..., только уже от nu1 и nu2')
write_obj(eq['diff(theta)_nu'], 'diff(theta)_nu', '3 уравнения связей на diff(theta) = ..., только уже от nu1 и nu2')

In [23]:
def dalamber_subs(obj):
    return lambda i: obj(i).subs(delta['psi'][i], eq['diff(psi)_nu'](i)).subs(delta['theta'][i], eq['diff(theta)_nu'](i)).subs(Derivative(psi[i], t), eq['diff(psi)'](i)).subs(Derivative(theta[i], t), eq['diff(theta)'](i))

In [24]:
#ТЕЛО 1 (платформа)
velocity[fkey(S)]        = lambda i: nu[1]*e['xi'] + nu[2]*e['eta']
F['platform']            = lambda i: zeros(3,1) # сил не действует
delta_r[fkey(S)]         = subs_delta(lambda i: delta['x']*e['x'] + delta['y']*e['y'])
J['platform']            = eye(3,3)*a  #  (симметричная) 
K['platform']            = lambda i: J['platform']*omega['platform'](i)
omega_delta['platform']  = subs_delta(lambda i: omega['platform'](i).subs(Derivative(alpha,t),delta['alpha']))
M['platform']            = lambda i: -W[i]*e['z']

A['platform'] = dalamber(mass = m['platform'],
                         velocity = velocity[fkey(S)],
                         F = F['platform'],
                         delta_r = delta_r[fkey(S)],
                         K = K['platform'],
                         M = M['platform'],
                         omega_delta = omega_delta['platform'])


In [25]:
#ТЕЛО 2 (вилки)
velocity[fkey(P)]    = lambda i: velocity[fkey(S)](i) + cross(omega['platform'](i), vec_by_2dots(S,P)(i))   # !!!!
# velocity['fork']   = v[fkey(P)]
F['fork']            = lambda i: zeros(3,1) # сил не действует
delta_r[fkey(P)]     = subs_delta(lambda i: delta_r[fkey(S)](i) + cross(omega['platform'](i), vec_by_2dots(S,P)(i)))
J['fork']            = zeros(3,3) # невесома
K['fork']            = lambda i: J['fork']*omega['fork'](i)
omega_delta['fork']  = subs_delta(lambda i: omega['fork'](i))
M['fork']            = lambda i: -(-W[i]*e['z']) -(-T[i]*n_wheel(i)) # magic

A['fork'] = dalamber(mass = 0,
                     velocity = velocity[fkey(P)],
                     F = F['fork'],
                     delta_r = delta_r[fkey(P)],
                     K = K['fork'],
                     M = M['fork'],
                     omega_delta = omega_delta['fork'])

In [26]:
#ТЕЛО 3 (колёса)
velocity[fkey(C)]     = lambda i: velocity[fkey(P)](i) + cross(omega['fork'](i), vec_by_2dots(P,C)(i))   # !!!!
F['wheel']            = lambda i: zeros(3,1) # сил не действует
delta_r[fkey(C)]      = subs_delta(lambda i: delta_r[fkey(P)](i) + cross(omega['fork'](i), vec_by_2dots(S,P)(i)))
J['wheel']            = Matrix([[b,0,0],[0,c,0],[0,0,b]])
K['wheel']            = lambda i: J['wheel'] * Matrix([scalar(omega['wheel'](i), e_wheel(i)), 
                                                       scalar(omega['wheel'](i), n_wheel(i)),
                                                       scalar(omega['wheel'](i), e['z'])])
omega_delta['wheel']  = subs_delta(lambda i: omega['wheel'](i))
M['wheel']            = lambda i: -(-W[i]*e['z']) -(-T[i]*n_wheel(i)) # magic

A['wheel'] = dalamber(mass= m['wheel'],
                     velocity = velocity[fkey(C)],
                     F = F['wheel'],
                     delta_r = delta_r[fkey(C)],
                     K = K['wheel'],
                     M = M['wheel'],
                     omega_delta = omega_delta['wheel'])


In [27]:
A['wheel'] = dalamber_subs(A['wheel'])
A['fork'] = dalamber_subs(A['wheel'])
A['platfrom'] = dalamber_subs(A['wheel'])

In [45]:
write_obj(A['wheel'],' A[wheel]', 'Даламбер для колеса')
write_obj(A['fork'],' A[fork]', 'Даламбер для вилки')
write_obj(A['platform'],' A[platform]', 'Даламбер для платформы')

In [34]:
# I have not the foggiest idea
# time ~1.5min

all_dalamber = 0
for i in range(3):
    all_dalamber += A['wheel'](i) + A['fork'](i) + A['platform'](i)


In [36]:
poly_dalamber = Poly(all_dalamber, [delta['x'], delta['y'], delta['alpha']])

In [43]:
# %%time

# s_dx = simplify(poly_dalamber.coeff_monomial(delta['x']))
# s_dy = simplify(poly_dalamber.coeff_monomial(delta['y']))
# s_dalpha = simplify(poly_dalamber.coeff_monomial(delta['alpha']))

Wall time: 9min 16s


In [37]:
# time ~10min

s_dx = simplify(poly_dalamber.coeff_monomial(delta['x']))
s_dy = simplify(poly_dalamber.coeff_monomial(delta['y']))
s_dalpha = simplify(poly_dalamber.coeff_monomial(delta['alpha']))

In [50]:
utils.write_obj(s_dx, 'dalamber_dx_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dx')
utils.write_obj(s_dy, 'dalamber_dy_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dy')
utils.write_obj(s_dalpha, 'dalamber_dalpha_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dAlpha')

In [51]:
coeff[delta['x']] = s_dx
coeff[delta['y']] = s_dy
coeff[delta['alpha']] = s_dalpha

In [52]:
# %%time

res_poly_dx = Poly(coeff[delta['x']], [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])
res_poly_dy = Poly(coeff[delta['y']], [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])
res_poly_dalpha = Poly(coeff[delta['alpha']], [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])


In [ ]:
# %%time
# res_poly_dx = Poly(coeff[delta['x']], [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])

In [54]:
# CHECK YOURSELF

print(pa(res_poly_dx))
print(pa(res_poly_dy))
print(pa(res_poly_dalpha))

{beta[1], m1, theta[2], beta[0], beta, Derivative(alpha(t), (t, 2)), theta[1], d, t, m2, theta[0], beta[2], alpha(t), Derivative(alpha(t), t), theta}
None
{beta[1], m1, theta[2], beta[0], beta, Derivative(alpha(t), (t, 2)), theta[1], d, t, m2, theta[0], beta[2], alpha(t), Derivative(alpha(t), t), theta}
None
{beta[1], beta[0], beta, b, theta[1], d, beta[2], m2, theta[0], theta, theta[2], a, W[1], Derivative(alpha(t), (t, 2)), W[2], t, W, alpha(t), Derivative(alpha(t), t), W[0]}
None


In [55]:
# CHECK YOURSELF

print(len(res_poly_dx.coeffs()))
print(len(res_poly_dy.coeffs()))
print(len(res_poly_dalpha.coeffs()))

4
4
4


In [56]:
res_poly_dx

Poly((3*m1*cos(alpha(t)) + 6*m2*cos(alpha(t)))*Derivative(nu1(t), t) + (-3*m1*sin(alpha(t)) - 6*m2*sin(alpha(t)))*Derivative(nu2(t), t) + (-2*d*m2*sin(alpha(t) + theta[0]) - 2*d*m2*sin(alpha(t) + theta[1]) - 2*d*m2*sin(alpha(t) + theta[2]) - 2*m2*sin(alpha(t) + beta[0]) - 2*m2*sin(alpha(t) + beta[1]) - 2*m2*sin(alpha(t) + beta[2]))*Derivative(alpha(t), (t, 2)) - 2*d*m2*cos(alpha(t) + theta[0])*Derivative(alpha(t), t)**2 - 2*d*m2*cos(alpha(t) + theta[1])*Derivative(alpha(t), t)**2 - 2*d*m2*cos(alpha(t) + theta[2])*Derivative(alpha(t), t)**2 - 3*m1*nu1(t)*sin(alpha(t))*Derivative(alpha(t), t) - 3*m1*nu2(t)*cos(alpha(t))*Derivative(alpha(t), t) - 6*m2*nu1(t)*sin(alpha(t))*Derivative(alpha(t), t) - 6*m2*nu2(t)*cos(alpha(t))*Derivative(alpha(t), t) + 2*m2*sin(alpha(t) + theta[0])*sin(beta[0] - theta[0])*Derivative(alpha(t), t)**2 + 2*m2*sin(alpha(t) + theta[1])*sin(beta[1] - theta[1])*Derivative(alpha(t), t)**2 + 2*m2*sin(alpha(t) + theta[2])*sin(beta[2] - theta[2])*Derivative(alpha(t), t)*

In [57]:
res_poly_dy

Poly((3*m1*sin(alpha(t)) + 6*m2*sin(alpha(t)))*Derivative(nu1(t), t) + (3*m1*cos(alpha(t)) + 6*m2*cos(alpha(t)))*Derivative(nu2(t), t) + (2*d*m2*cos(alpha(t) + theta[0]) + 2*d*m2*cos(alpha(t) + theta[1]) + 2*d*m2*cos(alpha(t) + theta[2]) + 2*m2*cos(alpha(t) + beta[0]) + 2*m2*cos(alpha(t) + beta[1]) + 2*m2*cos(alpha(t) + beta[2]))*Derivative(alpha(t), (t, 2)) - 2*d*m2*sin(alpha(t) + theta[0])*Derivative(alpha(t), t)**2 - 2*d*m2*sin(alpha(t) + theta[1])*Derivative(alpha(t), t)**2 - 2*d*m2*sin(alpha(t) + theta[2])*Derivative(alpha(t), t)**2 + 3*m1*nu1(t)*cos(alpha(t))*Derivative(alpha(t), t) - 3*m1*nu2(t)*sin(alpha(t))*Derivative(alpha(t), t) + 6*m2*nu1(t)*cos(alpha(t))*Derivative(alpha(t), t) - 6*m2*nu2(t)*sin(alpha(t))*Derivative(alpha(t), t) - 2*m2*sin(alpha(t) + beta[0])*Derivative(alpha(t), t)**2 - 2*m2*sin(alpha(t) + beta[1])*Derivative(alpha(t), t)**2 - 2*m2*sin(alpha(t) + beta[2])*Derivative(alpha(t), t)**2 - 2*m2*sin(beta[0] - theta[0])*cos(alpha(t) + theta[0])*Derivative(alpha(t

In [58]:
res_poly_dalpha

Poly((-4*m2*sin(beta[0]) - 4*m2*sin(beta[1]) - 4*m2*sin(beta[2]))*Derivative(nu1(t), t) + (4*m2*cos(beta[0]) + 4*m2*cos(beta[1]) + 4*m2*cos(beta[2]))*Derivative(nu2(t), t) + (3*a + 6*b + 4*d*m2*cos(beta[0] - theta[0]) + 4*d*m2*cos(beta[1] - theta[1]) + 4*d*m2*cos(beta[2] - theta[2]) + 12*m2)*Derivative(alpha(t), (t, 2)) + 6*d*m2*sin(beta[0] - theta[0])*Derivative(alpha(t), t)**2 + 6*d*m2*sin(beta[1] - theta[1])*Derivative(alpha(t), t)**2 + 6*d*m2*sin(beta[2] - theta[2])*Derivative(alpha(t), t)**2 + 2*m2*nu1(t)*sin(beta[0] - theta[0])*sin(theta[0])*Derivative(alpha(t), t) + 2*m2*nu1(t)*sin(beta[1] - theta[1])*sin(theta[1])*Derivative(alpha(t), t) + 2*m2*nu1(t)*sin(beta[2] - theta[2])*sin(theta[2])*Derivative(alpha(t), t) + 4*m2*nu1(t)*cos(beta[0])*Derivative(alpha(t), t) + 4*m2*nu1(t)*cos(beta[1])*Derivative(alpha(t), t) + 4*m2*nu1(t)*cos(beta[2])*Derivative(alpha(t), t) + 2*m2*nu2(t)*sin(beta[0])*Derivative(alpha(t), t) + 2*m2*nu2(t)*sin(beta[1])*Derivative(alpha(t), t) + 2*m2*nu2(t)*s

In [59]:
# система уравнений left*[nu1', nu2', alpha''] = right

left = Matrix([res_poly_dx.coeffs()[:3],
               res_poly_dy.coeffs()[:3],
               res_poly_dalpha.coeffs()[:3]])
right = Matrix([res_poly_dx.coeffs()[-1],
                res_poly_dy.coeffs()[-1],
                res_poly_dalpha.coeffs()[-1]])

In [60]:
Derivative(nu[1], t)

Derivative(nu1(t), t)

In [61]:
Derivative(nu[2], t)

Derivative(nu2(t), t)

In [62]:
Derivative(alpha, t, 2)

Derivative(alpha(t), (t, 2))

In [63]:
left_inv = left.inv() 

In [64]:
# %%time

res = left_inv * right

In [67]:
write_obj(res, 'vector_res_diff_eq_for', 'Вектор правых частей дифф уравнений на diff(nu1), diff(nu2), diff2(alpha)')

In [68]:
pa(res[0]) # nu1'

{beta[1], W, beta[0], beta, b, theta[1], d, m2, beta[2], theta[0], theta, m1, theta[2], a, W[2], t, W[1], alpha(t), Derivative(alpha(t), t), W[0]}


In [69]:
pa(res[1]) # nu2'

{beta[1], beta[0], beta, b, theta[1], d, m2, beta[2], theta[0], theta, m1, theta[2], a, W[1], W[2], t, W, alpha(t), Derivative(alpha(t), t), W[0]}


In [70]:
pa(res[2]) # alpha''

{beta[1], W, beta[0], beta, b, theta[1], d, m2, beta[2], theta[0], theta, m1, theta[2], a, W[2], t, W[1], alpha(t), Derivative(alpha(t), t), W[0]}


In [ ]:
%%time

simple_res = simplify(res[0])

In [ ]:
simplify(utils.subs_init(res))

In [ ]:
# ____________________________________________END___________________________________________

In [ ]:
# Полный Д'Аламбер Лагранжа

# A_full = 0
# A_full  = lambda i: A['platform'](i) + A['fork'](i) + A['wheel'](i)
# A_full_poly = lambda i: Poly(A_full(i).subs(delta['psi'][i], eq['diff(psi)'](i)).subs(delta['theta'][i], eq['diff(theta)'](i)), 
#     [delta['x'], delta['y'], delta['alpha']])

In [ ]:
# Полный Д'Аламбер для всего тела, просуммированный 

A_full_poly_sum = 0
for k in range(3):
    A_full_poly_sum += A_full_poly(k)

print_all_variables(A_full_poly_sum)

In [ ]:
print_all_variables(A_full_poly_sum)

In [ ]:
# Коэффиценты
# A_coeffs = lambda i: A_full_poly(i).coeffs()

In [ ]:
# Здесь будут лежать коэффиценты при dx dy dalpha  для всех 3 колёс
# ПОКА ДЛЯ ОДНОГО КОЛЕСА
#var = A_coeffs(0)

# coeff_wheel.append(A_coeffs(1))
# coeff_wheel.append(A_coeffs(2))

In [ ]:
#l = 0
#var[0] = var[0].subs({alpha: 0, beta[l]: 0, theta[l]: 0, nu[1]: 0, nu[2]: 0})
#var[1] = var[1].subs({alpha: 0, beta[l]: 0, theta[l]: 0, nu[1]: 0, nu[2]: 0})
#var[2] = var[2].subs({alpha: 0, beta[l]: 0, theta[l]: 0, nu[1]: 0, nu[2]: 0}) 

In [ ]:
coeff[delta['x']]     = lambda i: simplify(A_coeffs(i)[0])#.subs({alpha: 0, beta[i]: 0, theta[i]: 0, nu[1]: 0, nu[2]: 0})
coeff[delta['y']]     = lambda i: simplify(A_coeffs(i)[1])#.subs({alpha: 0, beta[i]: 0, theta[i]: 0, nu[1]: 0, nu[2]: 0})
coeff[delta['alpha']] = lambda i: simplify(A_coeffs(i)[2])#.subs({alpha: 0, beta[i]: 0, theta[i]: 0, nu[1]: 0, nu[2]: 0})

In [ ]:
coeff_dx ={}
coeff_dy ={}
coeff_dalpha ={}

In [ ]:
%%time
#коэффиценты для колеса при dx dy dalpha, просто в отдельные переменные
coeff_dx[0] = coeff[delta['x']](0)
coeff_dy[0] = coeff[delta['y']](0)
coeff_dalpha[0] = coeff[delta['alpha']](0)

In [ ]:
coeff_dalpha[0]

In [ ]:
%%time
#коэффиценты для колеса при dx dy dalpha, просто в отдельные переменные
coeff_dx[1] = coeff[delta['x']](1)
coeff_dy[1] = coeff[delta['y']](1)
coeff_dalpha[1] = coeff[delta['alpha']](1)

In [ ]:
%%time
#коэффиценты для колеса при dx dy dalpha, просто в отдельные переменные
coeff_dx[2] = coeff[delta['x']](2)
coeff_dy[2] = coeff[delta['y']](2)
coeff_dalpha[2] = coeff[delta['alpha']](2)

In [ ]:
# надеюсь нормально их суммировать сейчас
coeff_dx['sum'] = 0
coeff_dy['sum'] = 0
coeff_dalpha['sum'] = 0

for k in range(3):
    coeff_dx['sum'] += coeff_dx[k]
    coeff_dy['sum'] += coeff_dy[k]
    coeff_dalpha['sum'] += coeff_dalpha[k]

In [ ]:
# говорим что получившиеся выражения это полином, зависящий от nu1, nu2, (alpha)'' 
eq_by_dx     = Poly(coeff_dx[0],     [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])
eq_by_dy     = Poly(coeff_dy[0],     [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])
eq_by_dalpha = Poly(coeff_dalpha[0], [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])

In [ ]:
len(eq_by_dx.coeffs())

In [ ]:
len(eq_by_dy.coeffs())

In [ ]:
# посмотреть что получилось
len(eq_by_dx.coeffs())

In [ ]:
# Пример группировки
print(collect(eq_by_dx,
        [Derivative(nu[1], t), Derivative(nu[2], t), Derivative(alpha, t,2)]))

In [ ]:
# система уравнений left*[nu1', nu2', alpha''] = right

left = Matrix([eq_by_dx.coeffs()[:3],
               eq_by_dy.coeffs()[:3],
               eq_by_dalpha.coeffs()[:3]])
right = Matrix([eq_by_dx.coeffs()[-1],
                eq_by_dy.coeffs()[-1],
                eq_by_dalpha.coeffs()[-1]])

In [ ]:
alpha_t, nu1_t, nu2_t = symbols('alpha_t, nu1_t, nu2_t')
theta0, theta1, theta2 = symbols('theta0, theta1, theta2')
alpha1, alpha2 = symbols('alpha1, alpha2')

In [ ]:
def subs_for_ode(eq):
    eq = eq.subs({
            nu[1]: nu1_t, nu[2]: nu2_t,
            theta[0]: theta0, theta[1]: theta1, theta[2]: theta2,
            alpha: alpha1, Derivative(alpha, t): alpha2
    })
    return eq

In [ ]:
a = []
a += A_full_poly_sum.find(alpha)
a


In [ ]:
left = subs_init(left)
right = subs_init(right)

In [ ]:
print(left)

In [ ]:
%%time

left_inv = left.inv() # left^(-1)

In [ ]:
# Какие то варианты, которые наверно делают тоже самое
# from sympy.solvers.solveset import linsolve
# from sympy.solvers.solvers import solve_linear_system_LU

In [ ]:
eq['diff_eq'] = left_inv*right

In [ ]:
len(eq['diff_eq']) # nu1', nu2', alpha''

In [ ]:
eq['diff_eq']

In [ ]:
len(eq['diff_eq'])

In [ ]:
eq['psi']   = []
eq['theta'] = []

for k in range(3):
    eq['theta'].append(subs_init(eq['diff(theta)_nu'](k)))
    eq['psi'].append(subs_init(eq['diff(psi)_nu'](k)))

In [ ]:
# уравнения с 6 по 10

for k in range(3):
    eq['theta'][k] = subs_for_ode(eq['theta'][k])
    eq['psi'][k] = subs_for_ode(eq['psi'][k])
    
#####################
# константы  подставлены
#####################

In [ ]:
eq['psi'][0]

In [ ]:
for k in range(3):
    eq['diff_eq'] = eq['diff_eq'].subs({
        Derivative(theta[k], t): eq['theta'][k]
    })

In [ ]:
# уравнения с 1 по 3, с подставленными theta[i]'
T0, W0 = symbols('T0, W0')
eq['diff_eq'] = subs_for_ode(eq['diff_eq']).subs({
    W[0]: W0,
    T[0]: T0
})[2]

In [ ]:
dict_eq = {}
list_eq = [0,0,0,0,0,0,0,0,0,0]

list_eq[0] = dict_eq['nu1'] = eq['diff_eq'][0]
list_eq[1] = dict_eq['nu2'] = eq['diff_eq'][1]
list_eq[2] = dict_eq['alpha2'] = eq['diff_eq'][2]
list_eq[3] = dict_eq['alpha1'] = alpha2
list_eq[4] = dict_eq['theta0'] = eq['theta'][0]
list_eq[5] = dict_eq['theta1'] = eq['theta'][1]
list_eq[6] = dict_eq['theta2'] = eq['theta'][2]
list_eq[7] = dict_eq['psi0'] = eq['psi'][0]
list_eq[8] = dict_eq['psi1'] = eq['psi'][1]
list_eq[9] = dict_eq['psi2'] = eq['psi'][2]

In [ ]:
from mpmath import *

mp.dps = 15;  # точность
mp.pretty = True

In [ ]:
list_eq[0]

In [ ]:
temp = lambdify([nu[1], nu[2], alpha1, alpha2, theta0, theta1, theta2, psi[0], psi[1], psi[2]], list_eq[0])

In [ ]:
#############################################
# Далее решение этой системы дифф уравнений 
#############################################

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from math import *

In [ ]:
type(list_eq[0])

In [ ]:
def f(y, t, T0, W0, d):
    '''Определение правой части системы лин уравнений'''
    nu1_t, nu2_t, alpha1, alpha2, theta0, theta1, theta2, psi0, psi1, psi2 = y
    return list_eq

In [ ]:
begin_v = [0,0,0,0,0,0,0,0,0,0]
t = np.linspace(0, 1, 51) # ну вот такое себе конечно
T0 = 1.0
W0 = 1.0
d = 0.5

In [ ]:
res = odeint(f, begin_v, t, args=(T0, W0, d,))

In [ ]:
list_res = []

for x in range(10):
    list_res.append(res[:,x])